<h2 style="font-weight: 700; text-align: center">Long Short Term Memory (LSTM)</h2>

#### Importing necessary packages/libraries

In [ ]:
import numpy
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.datasets import imdb
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.layers import Dense, LSTM, Embedding
from keras_preprocessing.sequence import pad_sequences

#### Importing and arranging data

In [ ]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [ ]:
print(len(X_train[1]))

<p style="font-size: 14px">Making each review of uniform length, 600 characters</p>

In [ ]:
max_review_length = 600
X_train = pad_sequences(X_train, maxlen=max_review_length)
X_test = pad_sequences(X_test, maxlen=max_review_length)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


#### Creating the model

In [ ]:
embedding_vector_length = 64

model = Sequential([
    Embedding(top_words + 1, embedding_vector_length,
              input_length=max_review_length),
    LSTM(500),
    Dense(50, activation='relu'),
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(50, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['accuracy'])


In [ ]:
model.summary()

#### Model Fitting

In [ ]:
hist = model.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test))

#### Model Evaluation

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0) 

###### Validation Accuracy:

In [ ]:
print(scores[1])

##### Accuracy v/s Epochs

In [ ]:
plt.plot(hist.history['accuracy'], color='r')
plt.plot(hist.history['val_accuracy'], color='b')
plt.legend(['Training Accuracy', 'Testing Accuracy'])
plt.show()

#### Prediction

In [ ]:
y_pred = model.predict(X_test)
print(len(y_pred))

#### Prediction plot

In [ ]:
plt.figure(figsize=(15, 10))
plt.scatter(range(len(y_pred)), y_pred)
plt.show()


###### As it can be seen, most of the predictions lie at either 0 or 1, we can apply further thresholding to finalize the categories

##### Thresholding for 2 final categories

In [ ]:
for i in range(len(y_pred)):
    if (y_pred[i] >= 0.5):
        y_pred[i] = int(1)
    else:
        y_pred[i] = int(0)


##### Final evaluation

In [ ]:
incorr = 0
for i in range(len(y_pred)):
    if y_pred[i] != y_test[i]:
        incorr += 1

print(incorr, '/', len(y_pred))


In [ ]:
confuse_mat = tf.math.confusion_matrix(labels=y_pred, predictions=y_test)
plt.figure(figsize=(15, 8))
sns.heatmap(confuse_mat, annot=True, fmt='d')
plt.xlabel('Predicted Value')
plt.ylabel('True Value')
plt.show()
